In [1]:
%cd ..

/home/antonbabenko/Projects/newsclass01


In [2]:
%load_ext autoreload
%autoreload 2
%aimport src.model
%aimport src.data
%aimport src.train
%aimport src.utils
%aimport src.metrics

In [3]:
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import ttach as tta
import torch.nn as nn
import albumentations as A
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
from efficientnet_pytorch import EfficientNet
from pytorch_toolbelt import losses as L

from src.utils import set_seed, load_splits, get_tensorboard_writer
from src.data import ImageClassificationDataset
from src.train import fit
from src.metrics import f1_macro

In [4]:
SEED = 42
BATCH_SIZE = 32
EPOCHS = 40
LEARNING_RATE = 5e-3

In [5]:
set_seed(SEED)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
logs_dir = Path("logs") 
states_dir = Path("states") / "cv"
data_dir = Path("data")
folds = Path("folds") / "cv"
image_path = data_dir / "images" / "images"

In [8]:
splits = load_splits(folds, val_folds=[2], train_folds=[0, 1, 3, 4]) 

In [9]:
num_classes = splits["train"].source.nunique()

In [10]:
transform = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(),
    A.RandomRotate90(0.5),
    A.ColorJitter(),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2()
])
    
test_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2()
])

In [11]:
train_dataset = ImageClassificationDataset(
    df=splits["train"],
    folder=image_path, 
    mode="train",
    transform=transform
)

valid_dataset = ImageClassificationDataset(
    df=splits["val"],
    folder=image_path, 
    mode="val",
    transform=transform
)

# test_dataset = ImageClassificationDataset(
#     df=splits["test"],
#     folder=image_path, 
#     mode="test",
#     transform=test_transform
# )

In [12]:
# train_dataset = Subset(train_dataset, 20)
# valid_dataset = Subset(valid_dataset, 20)
# test_dataset = Subset(test_dataset, 20)

In [13]:
train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_data_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# test_data_loader = DataLoader(
#     dataset=test_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=False,
# )

In [14]:
dataloaders = {
    "train": train_data_loader,
    "val": val_data_loader,
#     "test": test_data_loader
}

In [15]:
mini_batch_data = next(iter(val_data_loader))

### Model Setup

In [16]:
model_name = 'efficientnet-b4'

In [17]:
model = EfficientNet.from_pretrained(model_name)

Loaded pretrained weights for efficientnet-b4


In [18]:
num_ftrs = model._fc.in_features
model._fc = torch.nn.Linear(num_ftrs, num_classes)
model = model.to(device)

### Metrics

In [19]:
metrics = {
    "f1_macro": f1_macro
}

### Tensorboard setup

In [20]:
model_name = f"{model_name}-{int(datetime.datetime.now().timestamp())}"
writer = get_tensorboard_writer(logs_dir, model_name) 

In [21]:
# criterion = nn.CrossEntropyLoss().to(device)
criterion = L.FocalLoss().to(device)
params = list(model.parameters())
optimizer = torch.optim.Adam(params, lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)

In [22]:
fit(
    model,
    criterion,
    optimizer,
    dataloaders,
    device,
    scheduler=scheduler,
    metrics=metrics,
    epochs=EPOCHS,
    model_name=model_name,
    model_folder=states_dir,
    writer=writer,
    fit_type="cv"
)

Epoch: 1/40, validation phase: 100%|██████████| 154/154 [00:54<00:00,  2.83it/s]


{'f1_macro': 0.07454929617086377}
Epoch: 1/40, time: 478.8328278064728 train loss: 0.7494287510996356, val loss: 0.8331470197832523


Epoch: 2/40, train phase:   0%|          | 0/616 [00:00<?, ?it/s]

Checkpoint was saved


Epoch: 2/40, validation phase: 100%|██████████| 154/154 [00:53<00:00,  2.88it/s]


...

Checkpoint was saved


Epoch: 16/40, train phase:   0%|          | 0/616 [00:00<?, ?it/s]

{'f1_macro': 0.3770447262939835}
Epoch: 15/40, time: 476.39202404022217 train loss: 0.4131841394730501, val loss: 0.4243992874283476


Epoch: 16/40, train phase:  42%|████▏     | 261/616 [02:57<04:01,  1.47it/s]


KeyboardInterrupt: 

In [ ]:
tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform())